In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import json
from tqdm import tqdm
import pandas as pd

import os
import sys
sys.path.append('..')
from utils import nethook
from utils import model_utils

In [3]:
MODEL_NAME = "gpt2-medium"  # gpt2-{medium,large,xl} or EleutherAI/gpt-j-6B
mt = model_utils.ModelAndTokenizer(MODEL_NAME, low_cpu_mem_usage=False)

model = mt.model
tokenizer = mt.tokenizer
tokenizer.pad_token = tokenizer.eos_token

In [4]:
prompt = [
    "This was an awesome movie",
    "This was a bad movie",
    "This was not a good movie",
    "That movie was garbage",
    "Iphone 7 is not a good phone",
    "Google new line of pixels are great"
]

prompt = ["<REVIEW>: " + p + " <SENTIMENT>" for p in prompt]

txt, ret_dict = model_utils.generate_fast(
    model, tokenizer,
    prompt,
    argmax_greedy = True,
    max_out_len= 40,
    # debug=True,
    get_answer_tokens=True,

    # track_interesting_words = [
    #     [" positive", " negative"],
    #     [" positive", " negative"]
    # ]
)

model_utils.print_formatted_results(prompt, txt, ret_dict)

<REVIEW>: This was an awesome movie <SENTIMENT>
<REVIEW>: This was an awesome movie <SENTIMENT> <REVIEW>: I love this movie <REVIEW>: I love this movie <REVIEW>: I love this movie <
p(answer):  p(' <'[1279])=0.0664, p('
'[198])=0.0569, p(' :'[1058])=0.05, p(' I'[314])=0.0458, p(' It'[632])=0.0208

<REVIEW>: This was a bad movie <SENTIMENT>
<REVIEW>: This was a bad movie <SENTIMENT> <REVIEW>: I don't know what to say <REVIEW>: I'm not sure what to say <REVIEW
p(answer):  p(' <'[1279])=0.0586, p(' :'[1058])=0.0528, p('
'[198])=0.0445, p(' I'[314])=0.0341, p(' ('[357])=0.0278

<REVIEW>: This was not a good movie <SENTIMENT>
<REVIEW>: This was not a good movie <SENTIMENT> <REVIEW>: I don't think it was a good movie <REVIEW>: I think it was a good movie
p(answer):  p(' <'[1279])=0.0508, p(' :'[1058])=0.0455, p(' I'[314])=0.0397, p('
'[198])=0.0332, p(' ('[357])=0.0229

<REVIEW>: That movie was garbage <SENTIMENT>
<REVIEW>: That movie was garbage <SENTIMENT> <REVIEW>: I'm not going to watch 

In [ ]:
checkpoint = torch.load('Saved_weights/Fine-Tuned_CLF__IMDB_50K/gpt2-medium/finetuned__epoch_2.pth')
print("loaded checkpoint")
model.load_state_dict(checkpoint)
print("loaded model weights")

loaded checkpoint
loaded model weights


In [ ]:
prompt = [
    "This was an awesome movie",
    "This was a bad movie",
    "This was not a good movie",
    "That movie was garbage",
    "Iphone 7 is not a good phone",
    "Google new line of pixels are great"
]

prompt = ["<REVIEW>: " + p + " <SENTIMENT>" for p in prompt]

txt, ret_dict = model_utils.generate_fast(
    model, tokenizer,
    prompt,
    argmax_greedy = True,
    max_out_len= 40,
    # debug=True,
    get_answer_tokens=True,

    # track_interesting_words = [
    #     [" positive", " negative"],
    #     [" positive", " negative"]
    # ]
)

model_utils.print_formatted_results(prompt, txt, ret_dict)

<REVIEW>: This was an awesome movie <SENTIMENT>
<REVIEW>: This was an awesome movie <SENTIMENT> positive positive negative positive positive positive positive positive positive positive positive positive positive positive positive positive positive positive positive positive positive positive positive positive positive
p(answer):  p(' positive'[3967])=0.9382, p(' negative'[4633])=0.0617, p(' Positive'[33733])=0.0, p(' positives'[38548])=0.0, p('positive'[24561])=0.0

<REVIEW>: This was a bad movie <SENTIMENT>
<REVIEW>: This was a bad movie <SENTIMENT> negative positive positive positive positive positive positive positive positive positive positive positive positive positive positive positive positive positive positive positive positive positive positive positive positive
p(answer):  p(' negative'[4633])=0.8916, p(' positive'[3967])=0.1082, p(' Negative'[36183])=0.0, p(' negatives'[42510])=0.0, p(' Positive'[33733])=0.0

<REVIEW>: This was not a good movie <SENTIMENT>
<REVIEW>: This wa